In [1]:
import torchvision
from torchvision import transforms

from torch.utils.data import DataLoader

In [2]:
from matplotlib.pyplot import imshow
%matplotlib inline

In [3]:
!ls data/custom_data/

raw_data  train_data


In [40]:
trans = transforms.Compose([
    transforms.Resize((64,128))
])

train_data = torchvision.datasets.ImageFolder(root='data/custom_data/raw_data/', transform=trans)

In [41]:
for num, (data, label) in enumerate(train_data):
    print(data, label)
#     imshow(data)
    
    if(label == 1):
        data.save('data/custom_data/train_data/scene/{}_{}.jpg'.format(num, label))
    else:
        data.save('data/custom_data/train_data/portrait/{}_{}.jpg'.format(num, label))

<PIL.Image.Image image mode=RGB size=128x64 at 0x7F86B20DF3C8> 0
<PIL.Image.Image image mode=RGB size=128x64 at 0x7F86B20DF438> 0
<PIL.Image.Image image mode=RGB size=128x64 at 0x7F86B20DF2E8> 0
<PIL.Image.Image image mode=RGB size=128x64 at 0x7F86B20DF390> 0
<PIL.Image.Image image mode=RGB size=128x64 at 0x7F86B20DF4A8> 0
<PIL.Image.Image image mode=RGB size=128x64 at 0x7F86B2207D68> 1
<PIL.Image.Image image mode=RGB size=128x64 at 0x7F86B20DF400> 1
<PIL.Image.Image image mode=RGB size=128x64 at 0x7F86B20E7F28> 1
<PIL.Image.Image image mode=RGB size=128x64 at 0x7F86B20E7DD8> 1
<PIL.Image.Image image mode=RGB size=128x64 at 0x7F86B20DF470> 1


## train model with custom data

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dsets

In [58]:
device = 'cpu' # 'cpu' or 'cuda'?
torch.manual_seed(1)
# torch.cuda.manual_seed_all(1)

training_epochs = 50
batch_size = 2
learning_rate = 0.001

In [59]:
trans = transforms.Compose([
    transforms.ToTensor()
])

train_data = torchvision.datasets.ImageFolder(root='data/custom_data/train_data/', 
                                              transform=trans)

data_loader = torch.utils.data.DataLoader(dataset=train_data, 
                                          batch_size=batch_size, 
                                          shuffle=True, 
                                          drop_last=True)

In [60]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__() # CNN, self 안하면 학습이 안돼요? 아뇨 멀쩡히 돼요..
        #(1,28,28)
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=5, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        #(8,14,14)
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        #(64,7,7)
        self.fc = nn.Linear(16*13*29, 2, bias=True)
        
        nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out # 이거 안하면 또 안돌아가..

In [61]:
net = CNN().to(device)
test_input = (torch.Tensor(3,3,64,128)).to(device)
test_out = net(test_input)

In [62]:
loss_func = nn.CrossEntropyLoss().to(device)
# loss_func = nn.BCELoss().to(device)
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [63]:
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0.0
    for num, (imgs, labels) in enumerate(data_loader):
        imgs = imgs.to(device)
        labels = labels.to(device)
        
        out = net(imgs)
        loss = loss_func(out, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        avg_cost += loss / total_batch
        
    print('[Epoch:{}] cost = {}'.format(epoch, avg_cost))

[Epoch:0] cost = 0.7317471504211426
[Epoch:1] cost = 0.6577640771865845
[Epoch:2] cost = 0.5712885856628418
[Epoch:3] cost = 0.4488232135772705
[Epoch:4] cost = 0.31133317947387695
[Epoch:5] cost = 0.1864018738269806
[Epoch:6] cost = 0.07658257335424423
[Epoch:7] cost = 0.048014264553785324
[Epoch:8] cost = 0.022109946236014366
[Epoch:9] cost = 0.046676427125930786
[Epoch:10] cost = 0.012096637859940529
[Epoch:11] cost = 0.011128216050565243
[Epoch:12] cost = 0.0023021474480628967
[Epoch:13] cost = 0.0032826457172632217
[Epoch:14] cost = 0.0022370803635567427
[Epoch:15] cost = 0.0007091190200299025
[Epoch:16] cost = 0.00048009888269007206
[Epoch:17] cost = 0.00028503540670499206
[Epoch:18] cost = 0.0002766147954389453
[Epoch:19] cost = 0.000259283056948334
[Epoch:20] cost = 0.0002476434165146202
[Epoch:21] cost = 0.00023943539417814463
[Epoch:22] cost = 0.00022766494657844305
[Epoch:23] cost = 0.00021765957353636622
[Epoch:24] cost = 0.00020581674471031874
[Epoch:25] cost = 0.000194569

## model save and load

In [64]:
torch.save(net.state_dict(), "./model/model.pth")

In [65]:
new_net = CNN().to(device)
new_net.load_state_dict(torch.load("./model/model.pth"))

<All keys matched successfully>

In [66]:
print(net.layer1[0])
print(new_net.layer1[0])

Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))


In [67]:
print(net.layer1[0].weight[0][0][0])
print(new_net.layer1[0].weight[0][0][0])

tensor([ 0.0800, -0.0312, -0.0076,  0.0680, -0.0973], grad_fn=<SelectBackward>)
tensor([ 0.0800, -0.0312, -0.0076,  0.0680, -0.0973], grad_fn=<SelectBackward>)


In [68]:
net.layer1[0].weight[0] == new_net.layer1[0].weight[0]

tensor([[[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]],

        [[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]],

        [[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]]])